# 📌 <span style="font-size:18px; color:#007acc;"><b>Introduction</b></span>

This is the second notebook in the series. The first notebook focused on data preprocessing and cleaning, including handling missing values, detecting and removing outliers, and performing exploratory data analysis (EDA). In this notebook, we continue from that point by importing necessary modules and defining custom functions and classes for model training and testing. A custom batch sampler is implemented for efficient data loading. For interpretability, SHAP analysis is performed to explain feature importance. The trained model is then saved for future use, and a FastAPI server is created to serve the model for local inference. Finally, predictions can be made by sending requests to the API. Note: This work was conducted on MS Azure, so some settings may need adjustments.



# <span style="font-size:18px; color:#007acc;"><b> Table of Contents</b></span>
1. [Install libraries](#Install-Libraries)
2. [Import modules](#Import-Modules) 
3. [Custom functions and classes for model training](#Custom-Functions-and-Classes-for-Model-Training)  
4. [Execution code for model training](#Execution-Code-for-Model-Training)  
5. [SHAP analysis](#SHAP-Analysis)
6. [Creating FAST API](#Create-FAST-API)
7. [Inference from API](#Inference-from-API)

## <span style="font-size:18px; color:#007acc;"><b> 1. Install libraries <a id="Install libraries"></a> ##

In [ ]:
# Required libraries to be installed
!pip install numpy 
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install azureml-core  # AzureML Core for interacting with Azure Machine Learning services
!pip install azure-storage-blob  # Azure Storage Blob for managing files in Azure Blob Storage
!pip install azureml-dataset-runtime  # AzureML Dataset Runtime for working with datasets in Azure ML
!pip install torch torchvision torchmetrics  # PyTorch for deep learning, torchvision for computer vision, torchmetrics for metrics
!pip install scikit-learn pandas numpy matplotlib seaborn
!pip install pyyaml
!pip install mlflow  # MLflow for managing machine learning workflows
!pip install fastapi uvicorn  # FastAPI for building web APIs
!pip install fsspec  # fsspec for working with file systems

## <span style="font-size:18px; color:#007acc;"><b> 2. Import modules <a id="Import modules"></a> ##

In [ ]:
# Standard Library
import sys
import math
import logging
import datetime
import threading
import requests
import yaml

# PyTorch and Related
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Sampler, random_split, TensorDataset
from torchvision import datasets, transforms, models
from torchmetrics import Precision, Recall, F1Score

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

# Plotting and Visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Data Handling
import numpy as np
import pandas as pd

# API
from fastapi import FastAPI
from pydantic import BaseModel
from uvicorn import run

# Remote File Access
import fsspec


## <span style="font-size:18px; color:#007acc;"><b> 3. Custom functions and classes for model training <a id="Custom functions and classes for model training"></a> ##

In [ ]:

"""
This code defines a custom PyTorch dataset, neural network model, and several utility functions for training, validation, and evaluation of a binary classification model.

1. `LoadDataset`: A custom dataset class to load tabular data from feature and label files, with support for YAML-based configuration.
2. `NNModel`: A neural network model with customizable layers, weight initialization, and dropout.
3. `ProportionalBatchSampler`: A custom batch sampler to ensure proportional sampling based on state labels.
4. Functions for training, validation, accuracy computation, ROC curve computation, and precision-recall curve computation are also defined.

Each class and function is documented with arguments, return types, and functionality explained.

PEP 8 recommendations have been followed, with the exception of line length exceeding 79 characters in some instances. 
"""


class LoadDataset(Dataset):
    """
    Custom PyTorch Dataset for loading tabular data from feature and label files,
    optionally using a YAML config for cleaner initialization.
    """

    def __init__(self, feature_file=None, label_file=None, sep='\t', skiprows=1, config_path=None):
        """
        Initializes the dataset from file paths or a YAML config.

        Args:
            feature_file (str): Path to the feature file (CSV/TSV).
            label_file (str): Path to the label file (CSV/TSV).
            sep (str): Column separator in the file. Default is tab ('\t').
            skiprows (int): Number of rows to skip (usually header). Default is 1.
            config_path (str): Optional path to a YAML config file containing keys:
                               'feature_file', 'label_file', 'sep', and 'skiprows'.
        """
        if config_path:
            with open(config_path, 'r') as config_file:
                config = yaml.load(config_file, Loader=yaml.FullLoader)
            feature_file = config.get('feature_file')
            label_file = config.get('label_file')
            sep = config.get('sep', sep)
            skiprows = config.get('skiprows', skiprows)

        # Validate file paths
        if not feature_file or not label_file:
            raise ValueError("Both feature_file and label_file must be provided either directly or through the config file.")

        # Load data
        feature_data = pd.read_csv(feature_file, sep=sep, skiprows=skiprows)
        label_data = pd.read_csv(label_file, sep=sep, skiprows=skiprows)

        # Convert to PyTorch tensors
        self.X = torch.tensor(feature_data.values, dtype=torch.float32)
        self.Y = torch.tensor(label_data.values, dtype=torch.float32)
        self.n_samples = self.X.shape[0]

    def __getitem__(self, index):
        """
        Returns a single (feature, label) pair at the given index.
        """
        return self.X[index], self.Y[index]

    def __len__(self):
        """
        Returns the total number of samples.
        """
        return self.n_samples


class NNModel(nn.Module):
    """
    Neural network model with customizable hidden layers and initialization.
    """

    def __init__(self, input_size, hidden_sizes, output_size, initialization, dropout):
        """
        Initialize the neural network model.

        Args:
            input_size (int): Number of input features.
            hidden_sizes (list): List of hidden layer sizes.
            output_size (int): Number of output units.
            initialization (str): Type of weight initialization.
            dropout (float): Dropout probability.
        """
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.initialize_weights(self.fc1, initialization)

        self.hidden_layers = nn.ModuleList()
        for i in range(len(hidden_sizes) - 1):
            layer = nn.Linear(hidden_sizes[i], hidden_sizes[i + 1])
            self.hidden_layers.append(layer)
            self.initialize_weights(layer, initialization)

        self.fc_out = nn.Linear(hidden_sizes[-1], output_size)
        self.initialize_weights(self.fc_out, initialization)

    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        x = self.fc_out(x)
        return x

    def num_flat_features(self, x):
        """
        Compute number of features after flattening.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            int: Number of flattened features.
        """
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def initialize_weights(self, layer, initialization):
        """
        Initialize weights of a given layer.

        Args:
            layer (nn.Module): Layer to initialize.
            initialization (str): Initialization type.
        """
        if initialization == 'uniform':
            nn.init.uniform_(layer.weight, -0.1, 0.1)
            nn.init.constant_(layer.bias, 0)
        elif initialization == 'normal':
            nn.init.normal_(layer.weight, mean=0, std=0.01)
            nn.init.constant_(layer.bias, 0)
        elif initialization == 'xavier':
            nn.init.xavier_uniform_(layer.weight)
            nn.init.constant_(layer.bias, 0)
        elif initialization == 'he':
            nn.init.kaiming_uniform_(layer.weight, nonlinearity='relu')
            nn.init.constant_(layer.bias, 0)
        else:
            raise ValueError("Invalid initialization type. Choose from 'uniform', 'normal', 'xavier', or 'he'.")

    def print_output_layer_weights(self, epoch):
        """
        Print weights of the output layer at a given epoch.

        Args:
            epoch (int): Current training epoch.
        """
        print(f"Epoch {epoch}: fc_out")
        print(self.fc_out.weight.data)


class ProportionalBatchSampler(Sampler):
    """
    Custom batch sampler that ensures proportional sampling based on state labels.
    """

    def __init__(self, data_source, batch_size, state_start_idx, state_end_idx):
        """
        Initialize the batch sampler.

        Args:
            data_source (torch.utils.data.TensorDataset): Dataset to sample from.
            batch_size (int): Size of each batch.
            state_start_idx (int): Start index for state slice.
            state_end_idx (int): End index for state slice.
        """
        self.data_source = data_source
        self.batch_size = batch_size
        self.state_start_idx = state_start_idx
        self.state_end_idx = state_end_idx
        self.num_samples = len(data_source)
        self._prepare_indices()

    def _prepare_indices(self):
        """Prepare internal indices and proportions for sampling."""
        input_tensor = self.data_source.tensors[0]
        states_tensor = input_tensor[:, self.state_start_idx:self.state_end_idx]
        state_indices = torch.argmax(states_tensor, dim=1).numpy()

        self.state_to_indices = {}
        for idx, state in enumerate(state_indices):
            self.state_to_indices.setdefault(state, []).append(idx)

        for indices in self.state_to_indices.values():
            np.random.shuffle(indices)

        self.state_proportions = {
            state: len(indices) / self.num_samples
            for state, indices in self.state_to_indices.items()
        }

        self.state_pointers = {state: 0 for state in self.state_to_indices}
        self.num_batches = math.ceil(self.num_samples / self.batch_size)

    def __iter__(self):
        """Yield a batch of indices at each iteration."""
        for _ in range(self.num_batches):
            batch_indices = []

            for state, proportion in self.state_proportions.items():
                n = int(round(proportion * self.batch_size))
                start = self.state_pointers[state]
                end = min(start + n, len(self.state_to_indices[state]))
                batch_indices.extend(self.state_to_indices[state][start:end])
                self.state_pointers[state] = end

            if len(batch_indices) < self.batch_size:
                remaining = self.batch_size - len(batch_indices)
                all_indices = sum(self.state_to_indices.values(), [])
                np.random.shuffle(all_indices)
                batch_indices.extend(all_indices[:remaining])

            np.random.shuffle(batch_indices)
            yield batch_indices

    def __len__(self):
        """Return the number of batches."""
        return self.num_batches


def train_model(model, device, train_loader, optimizer, pos_weight):
    """
    Train the neural network on training data.

    Args:
        model (nn.Module): Neural network model.
        device (torch.device): Computation device.
        train_loader (DataLoader): Training data loader.
        optimizer (torch.optim.Optimizer): Optimizer.
        pos_weight (torch.Tensor): Weight for positive class.

    Returns:
        float: Average training loss per sample.
    """
    model = model.double()
    model.train()
    train_loss = 0.0
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    for data, target in train_loader:
        data, target = data.to(device).double(), target.to(device).double()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    return train_loss / len(train_loader.dataset)


def valid_model(model, device, valid_loader, pos_weight):
    """
    Evaluate the model on validation data.

    Args:
        model (nn.Module): Neural network model.
        device (torch.device): Computation device.
        valid_loader (DataLoader): Validation data loader.
        pos_weight (torch.Tensor): Weight for positive class.

    Returns:
        float: Average validation loss per sample.
    """
    model.eval()
    model = model.to(device).double()
    valid_loss = 0.0
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight).to(device).double()

    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device).double(), target.to(device).double()
            output = model(data).to(device).double()
            loss = criterion(output, target).to(device).double()
            valid_loss += loss.item()

    return valid_loss / len(valid_loader.dataset)


def compute_accuracy(model, data_loader, device):
    """
    Compute accuracy and classification metrics.

    Args:
        model (nn.Module): Neural network model.
        data_loader (DataLoader): Data loader.
        device (torch.device): Computation device.

    Returns:
        tuple: accuracy, precision, recall, f1, FP, FN, TP, TN, confusion matrix
    """
    model = model.to(device).float()
    model.eval()

    CM = torch.zeros(2, 2, dtype=torch.int32)
    correct = 0
    total = 0
    all_predictions = torch.tensor([], dtype=torch.float32, device=device)
    all_targets = torch.tensor([], dtype=torch.float32, device=device)

    precision = Precision(average='macro', num_classes=1, task='binary').to(device)
    recall = Recall(average='macro', num_classes=1, task='binary').to(device)
    f1_score = F1Score(average='macro', num_classes=1, task='binary').to(device)

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device).float(), target.to(device).float()
            output = model(data)
            predicted = (torch.sigmoid(output) >= 0.5).float()

            correct += (predicted == target).sum().item()
            total += target.size(0)

            all_predictions = torch.cat((all_predictions, predicted), dim=0)
            all_targets = torch.cat((all_targets, target), dim=0)

            CM += torch.tensor(confusion_matrix(target.cpu(), predicted.cpu(), labels=[0, 1]))

    accuracy = correct / total * 100
    precision.update(all_predictions, all_targets)
    recall.update(all_predictions, all_targets)
    f1_score.update(all_predictions, all_targets)

    return (
        accuracy,
        precision.compute().item(),
        recall.compute().item(),
        f1_score.compute().item(),
        CM[0][1].item(),  # False Positive
        CM[1][0].item(),  # False Negative
        CM[1][1].item(),  # True Positive
        CM[0][0].item(),  # True Negative
        CM
    )


def compute_roc(model, data_loader, device):
    """
    Compute ROC curve and AUC score.

    Args:
        model (nn.Module): Neural network model.
        data_loader (DataLoader): Data loader.
        device (torch.device): Computation device.

    Returns:
        tuple: fpr, tpr, thresholds, auc_score
    """
    model = model.to(device).float()
    model.eval()

    all_predictions = torch.tensor([], dtype=torch.float32, device=device)
    all_targets = torch.tensor([], dtype=torch.float32, device=device)

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device).float(), target.to(device).float()
            output = model(data)
            predicted = torch.sigmoid(output)

            all_predictions = torch.cat((all_predictions, predicted), dim=0)
            all_targets = torch.cat((all_targets, target), dim=0)

    fpr, tpr, thresholds = roc_curve(all_targets.cpu().numpy(), all_predictions.cpu().numpy())
    roc_auc = auc(fpr, tpr)

    print(f'Number of thresholds: {len(thresholds)}')
    return fpr, tpr, thresholds, roc_auc


def compute_prc(model, data_loader, device):
    """
    Computes the Precision-Recall Curve (PRC) and AUC for a binary classification model.

    Args:
        model (torch.nn.Module): Trained model for making predictions.
        data_loader (torch.utils.data.DataLoader): DataLoader for evaluation data.
        device (torch.device): Device (CPU or GPU) for model and data.

    Returns:
        tuple: Precision, recall, PRC AUC, and thresholds arrays.
    """
    model = model.to(device).float()
    model.eval()
    
    all_predictions = torch.tensor([], dtype=torch.float32, device=device)
    all_targets = torch.tensor([], dtype=torch.float32, device=device)

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device).float(), target.to(device).float()
            output = model(data)
            predicted = torch.sigmoid(output)

            all_predictions = torch.cat((all_predictions, predicted), dim=0)
            all_targets = torch.cat((all_targets, target), dim=0)

    precision, recall, thresholds = precision_recall_curve(all_targets.cpu().numpy(), all_predictions.cpu().numpy())
    prc_auc = auc(recall, precision)
    
    return precision, recall, prc_auc, thresholds


class MetricsLogger:
    """
    A class to log and plot training and validation metrics during model training.

    Attributes:
        metrics (list): List of metrics to be tracked (e.g., loss, accuracy, precision, etc.).
        train_{metric} (dict): Stores metric values for the training phase (indexed by epoch).
        valid_{metric} (dict): Stores metric values for the validation phase (indexed by epoch).

    Methods:
        update(epoch, phase, **kwargs): Updates metrics for a given epoch and phase ('train' or 'valid').
        get_metric(phase, metric): Retrieves the metric values for a specific phase and metric.
        plot_metric(metric): Plots the specified metric across epochs for both training and validation.
    """
    def __init__(self):
        # Define the list of metrics and their phases
        self.metrics = [
            'losses', 'accuracies', 'false_positives', 'false_negatives', 
            'fpr', 'tpr', 'roc_auc', 'precision', 'recall', 'prc_auc', 'thresholds'
        ]
        
        # Initialize dictionaries for each metric for both train and valid phases
        for metric in self.metrics:
            setattr(self, f"train_{metric}", {})
            setattr(self, f"valid_{metric}", {})

    def update(self, epoch, phase, **kwargs):
        """Update metrics at a given epoch and phase ('train' or 'valid')"""
        for key, value in kwargs.items():
            # Dynamically form the attribute name
            attr_name = f"{phase}_{key}"
            if hasattr(self, attr_name):
                getattr(self, attr_name)[epoch] = value
            else:
                print(f"Warning: Unknown metric {attr_name}")

    def get_metric(self, phase, metric):
        """Get a dictionary of the metric values"""
        attr_name = f"{phase}_{metric}"
        return getattr(self, attr_name, None)

    def plot_metric(self, metric):
        """Plot a metric across epochs"""
        train_metric = getattr(self, f"train_{metric}", None)
        valid_metric = getattr(self, f"valid_{metric}", None)

        plt.figure(figsize=(8, 5))
        if train_metric:
            plt.plot(train_metric.keys(), train_metric.values(), label='Train')
        if valid_metric:
            plt.plot(valid_metric.keys(), valid_metric.values(), label='Validation')
        plt.title(f'{metric.replace("_", " ").title()} over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric.replace("_", " ").title())
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()



## <span style="font-size:18px; color:#007acc;"><b> 4. Execution code for model training   <a id="Execution Code for Model Training"></a> ## 

In [ ]:
"""
This script is responsible for loading configuration files, setting up the dataset, 
and preparing it for training and evaluation in a machine learning pipeline using PyTorch. 

Key operations performed in the script:
1. **Configuration Loading**: Loads configurations from 'config.yaml' and 'train_config.yaml' for setting random seed, device preference, 
   and model-specific parameters such as epochs, input size, and batch size.
   
2. **Data Preprocessing**: 
   - Loads the dataset and splits it into training, validation, and test sets using stratified splitting.
   - Scales the dataset using a MinMaxScaler (can be extended to other scalers).
   - Converts the data into PyTorch tensors and creates corresponding `TensorDataset` objects for training, validation, and testing.

3. **Model and Training Setup**: 
   - Initializes the model, optimizer (Adam), learning rate scheduler (MultiStepLR), and loss function.
   - Configures batch size dynamically based on the given percentage of the training dataset.
   - Supports handling of class imbalance by adjusting positive weights based on class distribution.

4. **Training Loop**: 
   - Iterates through epochs, trains the model, computes accuracy, loss, false positives, false negatives, ROC, and PRC metrics for both training 
     and validation datasets, and logs them using the `MetricsLogger` class.
   - Updates the learning rate scheduler after each epoch.

5. **Evaluation**: 
   - Computes the ROC curve and PRC metrics at each epoch for both training and validation datasets.
   
6. **Metric Plotting**: 
   - After training, plots the metrics (accuracy, loss, false positives, false negatives, ROC AUC, PRC AUC) for visualization using the `plot_metric` method of the `MetricsLogger`.

"""

# Load config from YAML
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Set random seed for reproducibility
np.random.seed(config['random_seed'])
torch.manual_seed(config['random_seed'])

# Load dataset
dataset = LoadDataset(config_path='config.yaml')

# Split the dataset into features (X) and labels (Y)
X, Y = dataset.X, dataset.Y

# Perform initial split of the data into training and temporary sets, stratifying by Y
X_train, X_temp, Y_train, Y_temp = train_test_split(
    X, Y,
    test_size=config['test_size_initial'],
    random_state=config['random_seed'],
    stratify=Y
)

# Further split the temporary set into validation and test sets, stratifying by Y_temp
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp, Y_temp,
    test_size=config['test_size_final'],
    random_state=config['random_seed'],
    stratify=Y_temp
)

# Instantiate scaler (support only MinMaxScaler here, but can be extended)
if config['scaling'] == 'MinMaxScaler':
    sc = MinMaxScaler()
else:
    raise ValueError(f"Scaler '{config['scaling']}' is not supported.")

# Fit and transform the training data
X_train_normalized = sc.fit_transform(X_train)

# Transform the validation and test data using the same scaler
X_val_normalized = sc.transform(X_val)
X_test_normalized = sc.transform(X_test)

# Convert to PyTorch tensors
train_input_tensor = torch.from_numpy(X_train_normalized).float()
train_output_tensor = torch.from_numpy(Y_train).float()
valid_input_tensor = torch.from_numpy(X_val_normalized).float()
valid_output_tensor = torch.from_numpy(Y_val).float()
test_input_tensor = torch.from_numpy(X_test_normalized).float()
test_output_tensor = torch.from_numpy(Y_test).float()

# PyTorch train, validation, and test sets
train = TensorDataset(train_input_tensor, train_output_tensor)
valid = TensorDataset(valid_input_tensor, valid_output_tensor)
test = TensorDataset(test_input_tensor, test_output_tensor)

# Initialize the metrics logger
metrics_logger = MetricsLogger()

# Set device
if config['device_preference'] == 'auto':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    device = torch.device(config['device_preference'])
print(device)

##### Load training config from YAML #####
with open('train_config.yaml', 'r') as file:
    train_config = yaml.safe_load(file)['training']

# Extract config values
# Define the variable names you want to extract
config_keys = [
    'num_epochs', 'input_size', 'output_size', 'hidden_sizes', 'initialization',
    'dropout', 'learning_rate', 'weight_decay', 'pos_weights', 'batch_percentage',
    'scheduler.milestones', 'scheduler.gamma', 'state_start_index', 'state_end_index'
]

# Use a loop to extract and assign values dynamically
for key in config_keys:
    keys = key.split('.')  # Handle nested keys like scheduler.milestones
    value = train_config
    for k in keys:
        value = value[k]  # Traverse through nested dictionaries
    globals()[key.replace('.', '_')] = value  # Assign the value to a variable with the same name

# Calculate batch size
total_train_samples = len(train)
batch_size = int(total_train_samples * batch_percentage)
batch_size = max(1, batch_size)

# DataLoaders
train_sampler = ProportionalBatchSampler(train, batch_size=batch_size, state_start_idx, state_end_idx)
train_loader = DataLoader(train, batch_sampler=train_sampler)
valid_loader = DataLoader(valid, batch_size=batch_size, shuffle=False)

# Iterate through positive weights
for pos_weight_mul in pos_weights:
    pos_count = torch.sum(train_output_tensor == 1).item()
    neg_count = torch.sum(train_output_tensor == 0).item()

    if pos_count == 0:
        raise ValueError("No positive samples in the dataset, cannot compute pos_weight.")

    pos_weight = (neg_count / pos_count) * pos_weight_mul
    pos_weight = torch.tensor([pos_weight], device=device)

    # Initialize model
    model = NNModel(input_size, hidden_sizes, output_size, initialization, dropout).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=scheduler_milestones, gamma=scheduler_gamma)

    for epoch in range(1, num_epochs + 1):
        train_loss = train_model(model, device, train_loader, optimizer, pos_weight)
        metrics_logger.update(epoch, "train", loss=train_loss)

        train_accuracy, _, _, _, train_fp, train_fn, _, _, _ = compute_accuracy(model, train_loader, device)
        metrics_logger.update(epoch, "train", accuracy=train_accuracy, false_positives=train_fp, false_negatives=train_fn)

        valid_loss = valid_model(model, device, valid_loader, pos_weight)
        metrics_logger.update(epoch, "valid", loss=valid_loss)

        valid_accuracy, _, _, _, valid_fp, valid_fn, _, _, _ = compute_accuracy(model, valid_loader, device)
        metrics_logger.update(epoch, "valid", accuracy=valid_accuracy, false_positives=valid_fp, false_negatives=valid_fn)

        # Scheduler step
        scheduler.step()

    # Compute ROC curve
    train_fpr, train_tpr, train_roc_auc, train_thresholds = compute_roc(model, train_loader, device)
    valid_fpr, valid_tpr, valid_roc_auc, valid_thresholds = compute_roc(model, valid_loader, device)
    metrics_logger.update(epoch, "train", fpr=train_fpr, tpr=train_tpr, roc_auc=train_roc_auc, thresholds=train_thresholds)
    metrics_logger.update(epoch, "valid", fpr=valid_fpr, tpr=valid_tpr, roc_auc=valid_roc_auc, thresholds=valid_thresholds)

    # Compute PRC
    train_precision, train_recall, train_prc_auc, train_thresholds = compute_prc(model, train_loader, device)
    valid_precision, valid_recall, valid_prc_auc, valid_thresholds = compute_prc(model, valid_loader, device)
    metrics_logger.update(epoch, "train", precision=train_precision, recall=train_recall, prc_auc=train_prc_auc, thresholds=train_thresholds)
    metrics_logger.update(epoch, "valid", precision=valid_precision, recall=valid_recall, prc_auc=valid_prc_auc, thresholds=valid_thresholds)

# Plot the metrics
metrics_logger.plot_metric("accuracy")
metrics_logger.plot_metric("loss")
metrics_logger.plot_metric("false_positives")
metrics_logger.plot_metric("false_negatives")
metrics_logger.plot_metric("roc_auc")
metrics_logger.plot_metric("prc_auc")

## <span style="font-size:18px; color:#007acc;"><b> 5. Shap analysis   <a id="Shap analysis"></a> ## 

In [ ]:

"""
Performs SHAP (Shapley Additive Explanations) analysis on a machine learning model to 
interpret feature contributions to predictions. 

1. Computes SHAP values using SHAP's DeepExplainer.
2. Visualizes feature importance with a summary plot and a bar plot.

Requires a trained model and input data (`valid_input_tensor` and `train_input_tensor`).
"""

def shap_analysis(model, input_tensor):
    """Performs SHAP analysis using the DeepExplainer."""
    # Ensure model is in evaluation mode and on CPU
    model.eval()
    model = model.cpu()

    # Create SHAP explainer
    explainer = shap.DeepExplainer(model, input_tensor)

    # Compute SHAP values
    shap_values = explainer.shap_values(input_tensor, check_additivity=False)
    
    return shap_values


# Perform SHAP analysis
subset_tensor = valid_input_tensor[:10000]
model.eval()
model = model.cpu()
model_predictions = model(subset_tensor).detach().numpy()[:, 0]

shap_values = shap_analysis(model, subset_tensor)
shap_values = np.squeeze(shap_values)  # This will remove all dimensions of size 1

shap_values = np.array(shap_values)

# Sum over the feature dimension (axis=1)
instance_sum = np.sum(shap_values, axis=1)

# Generate the summary plot
shap.summary_plot(
    shap_values, 
    features=subset_tensor.cpu().numpy(), 
    feature_names=["Feature_" + str(i) for i in range(train_input_tensor.size(1))]
)

# Generate the bar plot
shap.plots.bar(shap_exp)


## <span style="font-size:18px; color:#007acc;"><b> 6. Creating FAST API <a id="Create FAST API"></a> ##    

In [ ]:
"""
This script serves a trained machine learning model as an API using FastAPI. The model is loaded from a configuration 
file, and the weights are loaded from a pre-trained model file. It accepts input data through HTTP POST requests, 
makes predictions using the loaded model, and returns the results.

Functions:
- load_model_config: Loads the model configuration (training parameters) from a YAML configuration file.
- run_app: Runs the FastAPI server in a separate thread to serve predictions.
- predict: A POST endpoint that takes input features, processes them, and returns the model's prediction.

Classes:
- InputData: A Pydantic model for validating the structure of the incoming data.

The FastAPI server listens on port 8000 and provides a `/predict/` endpoint for model inference. The server is run in 
a separate thread to allow asynchronous execution of requests.

Requirements:
- FastAPI for serving the model as an API.
- PyTorch for loading the model and making predictions.
- YAML for loading configuration from the 'config' file.
"""

# Load model configuration from the config file
def load_model_config():
    with open('config', 'r') as file:  # Load 'config' file
        config = yaml.safe_load(file)
    return config['training']

# Initialize FastAPI app
app = FastAPI()

# Define a Pydantic model for input data
class InputData(BaseModel):
    features: list

# Load model parameters from the config file
model_config = load_model_config()

input_size = model_config['input_size']
output_size = model_config['output_size']
hidden_sizes = model_config['hidden_sizes']
initialization = model_config['initialization']

# Create the model instance
model = NNModel(input_size, hidden_sizes, output_size, initialization)
model.load_state_dict(torch.load('model_weights.pth'))  # Load pre-trained weights
model.eval()  # Set the model to evaluation mode

@app.post("/predict/")
async def predict(data: InputData):
    # Convert input data to tensor and ensure it has the correct shape
    input_tensor = torch.tensor(data.features, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        prediction = model(input_tensor)
    return {"prediction": prediction.tolist()}

# Function to run the app in a separate thread
def run_app():
    run(app, host="0.0.0.0", port=8000, log_level="info")

# Start the FastAPI server in a separate thread
thread = threading.Thread(target=run_app)
thread.start()


## <span style="font-size:18px; color:#007acc;"><b> 7. Inference from API <a id="Inference from API"></a> ##    

In [ ]:
"""
This script extracts a single example from the test dataset, sends it to a FastAPI model server for prediction, and 
prints both the predicted and true labels. The steps include:

1. Extract the first example (feature set) from the input tensor and the corresponding true label.
2. Send a POST request to the FastAPI endpoint with the feature data.
3. Process the response, which is assumed to contain logits from the model, apply the sigmoid activation function 
   to get probabilities, and convert them into binary predictions (0 or 1).
4. Print the status code and response from the server, as well as the predicted labels and true label for comparison.

Error handling is included for failed requests and invalid JSON responses.

Dependencies:
- requests for making HTTP requests.
- The sigmoid function to convert logits into probabilities.
"""


# Extract the first example (row) from test_input_tensor and convert it to a list
features = test_input_tensor[:1].tolist()[0]  # Extracts the first example and converts it to a list

# Extract the true label corresponding to the first example
true_label = test_output_tensor[0].item()  # Converts the tensor value to a Python scalar

# Define the endpoint
url = "http://localhost:8000/predict/"

data = {"features": features}  # Use the extracted features

try:
    # Make the POST request
    response = requests.post(url, json=data)

    # Print status code
    print(f"Status Code: {response.status_code}")

    # Print response text for debugging
    print(f"Response Text: {response.text}")

    # Assuming the response is a JSON array of logits
    logits = response.json()

    # Apply the sigmoid function and threshold to each logit
    prediction = sigmoid(logits)
    prediction = [1 if logit >= 0.5 else 0 for logit in logits]

    # Print the resulting predictions (0 or 1)
    print("Predicted Labels:", prediction)

    # Print the true label
    print("True Label:", true_label)

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as e:
    print(f"JSON decode error: {e}")